# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from collections import Counter
from nltk.stem import SnowballStemmer
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
display(data.dtypes)



children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

**Датасет содержит информацию с данными банка или кредитной организации, содержит все виды файлов
  

Выведим часть датасета, чтоб посмотреть на него.


In [2]:
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим в каких столбца отсутствуют данные


In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Отсутствуют значения только в столбцах 'days_employed' и 'total_income'.
Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения (текущая точность нам не нужна)



Проверим, это одни и те же строки, где отсутствуют данные по обоим столбцам ('days_employed' и 'total_income') или нет.

In [4]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB



В строках, где отсутствуют данные в столбце 'days_employed', отсутствуют данные и по 'total_income'.

Проверим, из разных ли это профессий люди (если из одной, то можно будет заполнить NaN средним по этой группе).


In [5]:
print(data.isnull().mean())


children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


In [6]:
    (print(data[(data['total_income'].isnull() == True) & (data['days_employed']
        .isnull() == True)]['income_type'].value_counts()))

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Значения отсутствуют в столбцах total_income и days_employed.
Проверим это люди из разных сфер (прфессий или из одной), ели из одно, то можно весь столбец заменить на срдене арефметическое. 


Люди из разных групп, значит взять среднее по одной группе профессий ('income_type') уже нельзя, будем заполнять NaN по среднему из каждой группы.

In [7]:
from collections import Counter
data.describe( include='all')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


1. Минимальное значение по 'children' - -1. Вероятно, это просто отсутствующая информация (хотя это ведь кредиты), скорей всего неверный перенос
2. Максимальное значение по 'children' - 20. Выглядит не очень реалистично,  проверим сколько таких строк имеется в датафрейме.    Вполне возможно, что это ошибка.
2. Отрицательные значения по количеству отработанных дней 'days_employed' - вполне возможно, что если возьмем значения по модулю, то данные станут пригодными для работы.
3. Среднее значение по количеству отработанных дней 'days_employed'  63046 дней (173 года). Кто-то нам нарушает корректность картины.
4. Возраст по некоторым строкам 'dob_years' равен 0 - предполагаю, что просто не указан (хотя, опять же, это ведь кредиты!).



Нужно разобраться со всеми этими некорректными дынными


Выведем некоторые данные по столбцу 'days_employed' чтоб понять в чем ошибка по пункту 3 и 4 в гепотезе выше:

In [8]:

print("Количество строк с 'days_employed' > 0:", data[data['days_employed'] > 0].shape[0])
print("Количество строк с 'days_employed' < 0:", data[data['days_employed'] < 0].shape[0])
data_pens = data[(data['days_employed'] > 0) & (data['income_type'] == 'пенсионер')]
print("Количество пенсионеров с 'days_employed' > 0:", data_pens.shape[0])
print("Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20:", data_pens[data_pens['days_employed'] > 33000].shape[0])
print("Среднее количество отработанных дней среди пенсионеров:", data_pens['days_employed'].mean())

Количество строк с 'days_employed' > 0: 3445
Количество строк с 'days_employed' < 0: 15906
Количество пенсионеров с 'days_employed' > 0: 3443
Количество пенсионеров с 'days_employed' > 0 и кол-вом отработанных лет > 20: 3443
Среднее количество отработанных дней среди пенсионеров: 365003.4912448612



Выведем некоторые данные по столбцам 'dob_years' и 'children', чтоб разобраться с с пунктом 1 и 2

In [9]:

print("Количество людей с нулевым возрастом:", data[data['dob_years'] == 0].count()[0])
print("Количество людей с -1 ребенком:", data[data['children'] == -1].count()[0])
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0])
print("Количество уникальных людей с 20 детьми:", len(data[data['children'] == 20]['total_income'].unique()))

Количество людей с нулевым возрастом: 101
Количество людей с -1 ребенком: 47
Количество строк с 20 детьми: 76
Количество уникальных людей с 20 детьми: 68


1)  более 70% значений из 'days_employed' отрицательные и не относятся к пенсионерам - просто возьмем весь столбец по модулю, чтоб убрать;
2) остальные практически все значения > 0 (кроме двух) - это пенсионеры, причем по всем из них стаж работы >  90 лет, что,  выглядит не правдаподобным;
3) нулевой возраст для некоторых клиентов - предполагаю,что данные просто были не внесены в базу, необходимо исправлять;
4) "-1 ребенок" - к исправлению;
5) "20 детей" - это не уникальный случай, а просто ошибка (может быть, корректное значение - "2"). Тоже к исправлению.

1. Попарные NaN 'days_employed' и 'total_inome' заполним значением по данной группе 'type' в соотношении с возрастом.
2. Обрабатываем отрицательные значения по количеству отработанных дней 'days_employed'
3. Столбец 'education' имеет не однотипные значения (напр., 'Среднее'-'среднее'-'СРЕДНЕЕ') - необходимо будет привести к общему виду.
4. Исправляем минимальное значение по 'children' (-1) на 0, а также приведем в соответствие кол-во детей 20...
5. Обрабатываем нулевой возраст по некоторым строкам 'dob_years'.
6. 'days_employed' по пенсионерам > 90 лет - надо приводить к нормальным значениям.

## Шаг 2. Предобработка данных

In [10]:

grouped_data = data.groupby('income_type').agg({'days_employed': 'median', 'total_income': 'median'})
def mean_fill(row):
    if pd.notnull(row['total_income']):
        return row['total_income']
    else:
        return grouped_data[row['income_type']]['total_income']
print(grouped_data)

                 days_employed   total_income
income_type                                  
безработный      366413.652744  131339.751676
в декрете         -3296.759962   53829.130729
госслужащий       -2689.368353  150447.935283
компаньон         -1547.382223  172357.950966
пенсионер        365213.306266  118514.486412
предприниматель    -520.848083  499163.144947
сотрудник         -1574.202821  142594.396847
студент            -578.751554   98201.625314


### Обработка пропусков

Берем значения по 'days_employed' и 'total_income' по модулю.
Добавляем столбец 'ratio_days_employed' для вычисления коэффициента полноты количества отработанных дней (учитывая то, что официально можно трудиться с 16 лет), чтобы впоследствии заполнить NaN по 'days_employed' с учетом среднего этого показателя по группе.

In [11]:
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['ratio_days_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-16)*365)

data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


Заполним пропуски NaN в датафрейме data:
-'days_employed': NaN заменяем на произведение среднего коэффициента по группе 'income_type', умноженного на возраст в днях;
-'total_income': NaN заменяем на среднее значение по зарплате по группе 'income_type'.
удалим вспомогательный столбец


In [12]:
data['days_employed'] = data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
data = data.drop('ratio_days_employed', axis=1)
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Кол-во детей с отрицательным значением заменим на среднюю методом abs(), "ошибку" ввиде 20 детей, щаменим на 2

In [13]:
data['children'] = data['children'].abs()

data.loc[data['children']>10, 'children']=2

data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверяем данные

In [14]:
data.isnull().mean()

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

после проверки пропусков не выявлено

In [15]:
print(data[(data['total_income'].isnull() == True) & (data['days_employed'] \
    .isnull() == True)]['income_type'].value_counts())

Series([], Name: income_type, dtype: int64)


Проверили данные двумя методами, пропусков не осталось

Cмотрим количество нулей по возрасту (столбец 'dob_years') и количество несовершеннолетних


In [16]:
# смотрим количество нулей по возрасту (столбец 'dob_years') и количество несовершеннолетних
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])
print("Количество строк с 'dob_years' < 19 -", data[data['dob_years'] < 19].count()[0])

Количество строк с 'dob_years' 0 - 101
Количество строк с 'dob_years' < 19 - 101


Количество совпадает? это одни и те же люди. Заполним значения средним по каждой группе

In [17]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))



In [18]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,1.522834e+08,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,0.755528,1.078074e+09,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,3.336411e+05,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,5.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обработали пропуски, убрали некооректные данные по кол-ву детей, также откоррктировали отрицательный стаж, у нас остался неккоректный стаж по пенсионерам, причины этой ошибки выяснить тяжело, возможно при переносе данных или некорректное заполение. Если у нас нет возможности уточнить эти данные, можно предположить, что теоретически стаж пенсионеров должен быть больше,чем у остальных категорий(за исключением военных и структу МВД, но они редко выходят на пенсию с наступлением пенсионного возраста, а продолжают работать), поэто если нам это будет нужно этот столбец можно категоризировать например стаж: более 50 лет и все что выше попадет в эту категорию.
Думамое можно сделать вывод, что основные все явно некорректные значения откорректированы.

### Замена типа данных

In [19]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


месячную зп и стаж перевели в целые числа, таблица стала более читаемая, приведем столбец education к одному регистру, а то он "бросается" в глаза.

In [20]:
data['education'] = data['education'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Обработка дубликатов

In [21]:
data.duplicated().sum()

71

Посмотрим на дубликаты

In [22]:
data[data.duplicated(keep=False)].sort_values(by=['income_type', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
680,1,37228871,30,высшее,0,женат / замужем,0,F,госслужащий,0,170898,покупка жилья для семьи
18349,1,37228871,30,высшее,0,женат / замужем,0,F,госслужащий,0,170898,покупка жилья для семьи
9013,2,44674645,36,высшее,0,женат / замужем,0,F,госслужащий,0,170898,получение образования
14432,2,44674645,36,высшее,0,женат / замужем,0,F,госслужащий,0,170898,получение образования
8490,1,23891898,31,среднее,1,женат / замужем,0,F,компаньон,0,202417,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
12373,0,49251988,58,среднее,1,женат / замужем,0,M,сотрудник,0,161380,покупка недвижимости
15091,0,49251988,58,среднее,1,гражданский брак,1,M,сотрудник,0,161380,на проведение свадьбы
20662,0,49251988,58,среднее,1,гражданский брак,1,M,сотрудник,0,161380,на проведение свадьбы
554,0,50950332,60,среднее,1,женат / замужем,0,M,сотрудник,0,161380,покупка недвижимости


Похоже на бональное задвоение, удалим их.

In [23]:
data = data.drop_duplicates()
print(data.duplicated().sum())

0


Удалили из масива данных дубликаты

### Лемматизация

В стодбце purpose есть список на первый взгляд похожих целей, выведим список уникальных целей

In [24]:
unique_purposes = data['purpose'].value_counts().index.tolist()
data['purpose'].value_counts().to_frame()

,purpose
свадьба,791
на проведение свадьбы,768
сыграть свадьбу,765
операции с недвижимостью,675
покупка коммерческой недвижимости,661
операции с жильем,652
покупка жилья для сдачи,651
операции с коммерческой недвижимостью,650
покупка жилья,646
жилье,646


видно, что есть ряд одинаковых целей по разному интерпретированных, применим метод лемматизации, чтобы выделить уникальные леммы

In [25]:
from collections import Counter
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')

from pymystem3 import Mystem
m = Mystem()


In [26]:

def do_lemma(row):
    lemma = m.lemmatize(row)
    return lemma


Cоздадим дополнительный столбец с лемматизированными целями


In [27]:
data['purpose_lemma'] = data['purpose'].apply(do_lemma)

создаем строку из уникальных целей



In [28]:
all_lemmas_list = data['purpose_lemma'].values
flat_lemmas_list = []
for sublist in all_lemmas_list:
    for item in sublist:
        flat_lemmas_list.append(item)

In [29]:
print(all_lemmas_list)
from collections import Counter
#посчитаем самые популярные слова, чтобы обобщить цели кредитов, это: недвижимость, образование, автомобиль, свадьба
print('Посчитаем самые популярные леммы в графе purpose:')
print(Counter(flat_lemmas_list))

[list(['покупка', ' ', 'жилье', '\n'])
 list(['приобретение', ' ', 'автомобиль', '\n'])
 list(['покупка', ' ', 'жилье', '\n']) ... list(['недвижимость', '\n'])
 list(['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'])
 list(['на', ' ', 'покупка', ' ', 'автомобиль', '\n'])]
Посчитаем самые популярные леммы в графе purpose:
Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


выводим функцию для создания словаря

In [30]:
def dictionary(world, data):
    for row in data:
        if world == row:
            return row

print('Cоздадим словарь с категориями в графе purpose, вдальнейшем его можно будет пополнять:')
#создадим словарь с категориями, вдальнейшем его можно будет пополнять

dict = []
#вручную добавим выбранные мною слова
dict.append(dictionary('жилье', flat_lemmas_list))
dict.append(dictionary('недвижимость', flat_lemmas_list))
dict.append(dictionary('образование', flat_lemmas_list))
dict.append(dictionary('автомобиль', flat_lemmas_list))
dict.append(dictionary('свадьба', flat_lemmas_list))
print(dict)

Cоздадим словарь с категориями в графе purpose, вдальнейшем его можно будет пополнять:
['жилье', 'недвижимость', 'образование', 'автомобиль', 'свадьба']


Создаем столбец с категориями и функцию которая выдает категории

In [31]:
def category(data):
    if dict[0] in data:
        return dict[1]
    elif dict[1] in data:
        return dict[1]
    elif dict[2] in data:
        return dict[2]
    elif dict[3] in data:
        return dict[3]
    else:
        return dict[4]
    

data['category_purpose'] = data['purpose_lemma'].apply(category)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


In [32]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


Мы получили список уникальных лемм, недвижимость и жилье это одно и тоже.

### Категоризация данных

In [33]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children


,count_children,sum_children,result_children
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Вывод:
Так мы изучаем влияние возврата кредита в срок и наличие детей, видим отрицательное значение и значение 20, считаю, что это опечатка при переносе данных. Категоризация была применена вместе с лемматизацией выше в одной функции. Категоризовать количество детей нет необходимости, категориями будет как раз их количество, так как всего 6 вариантов.

</div>  Я не смог заменить 20 детей на медиану, использовал data['children'] = data['children'].replace(20, children_median), но выдает ошибку, не могу понять почему, тоже самое с -1, это опечатка, как бы вывод могу сделать итак, но хочется заменить эти данные </b>

In [34]:
def salary_cat(total_income):
    if total_income <=50000:
                    return 'бедный'
    elif 50000 <total_income<=120000:
                    return 'средний'
    elif 120000 <total_income<=250000:
                    return 'выше среднего'
    else:
                    return 'богатый'
salary_cat


<function __main__.salary_cat(total_income)>

In [35]:
data['salary_cat'] = data['total_income'].apply(salary_cat)
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_purpose,salary_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,богатый
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,богатый
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,богатый
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,выше среднего
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,выше среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,выше среднего


Категория бедный не выпадает если выводить хоть 10 первых, хоть 10 последних строк, выведим её отдельно, чтоб потом сделать вывод, на что берут кредит люди в этой категории.

In [36]:
data = data.set_index(['salary_cat'])
data.loc[data.index.isin(['бедный'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category_purpose
salary_cat,,,,,,,,,,,,,,
бедный,0,1181,54,среднее,1,женат / замужем,0,F,сотрудник,0,44591,автомобиль,"[автомобиль, \n]",автомобиль
бедный,0,338113,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929,автомобили,"[автомобиль, \n]",автомобиль
бедный,0,72,42,среднее,1,гражданский брак,1,F,компаньон,0,49463,сделка с подержанным автомобилем,"[сделка, , с, , подержанный, , автомобиль, \n]",автомобиль
бедный,0,382189,60,среднее,1,Не женат / не замужем,4,F,пенсионер,0,31534,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]",недвижимость
бедный,1,3597,39,среднее,1,женат / замужем,0,F,компаньон,0,40891,покупка своего жилья,"[покупка, , свой, , жилье, \n]",недвижимость
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
бедный,0,358631,71,среднее,1,женат / замужем,0,F,пенсионер,0,41411,профильное образование,"[профильный, , образование, \n]",образование
бедный,0,1446,45,среднее,1,женат / замужем,0,F,сотрудник,0,34472,покупка жилья для сдачи,"[покупка, , жилье, , для, , сдача, \n]",недвижимость
бедный,0,338060,59,среднее,1,женат / замужем,0,F,пенсионер,0,47388,получение образования,"[получение, , образование, \n]",образование


In [37]:
print('Создали словарь:')
family_dict = data[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)
print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = data.groupby('family_status_id')['debt'].agg(['count', 'sum', 'mean'])
display(a)
# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создали словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем




Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,mean
family_status_id,,,
0,12339,931,0.075452
1,4151,388,0.093471
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,mean
0,женат / замужем,12339,931,0.075452
1,гражданский брак,4151,388,0.093471
2,вдовец / вдова,959,63,0.065693
3,в разводе,1195,85,0.071130
4,Не женат / не замужем,2810,274,0.097509


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.097561
2,2128,202,0.094925
1,4855,445,0.091658
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


Вывод: кредитору, не имеющие детей менее склонны к просрочке по кредитам, видимо сказывается финансовая нагрузка. Но также видно,что люди у кого три ребенка реже просрачевают, чем у кого три ребенка, возможно еще влияет возвраст детей, но у нас нет этих данных.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = data.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = data.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод: семейный статус влияент на уплату кредита в срок, люди не сосотящие в браке или находящиеся в гражданском браке обладают более худщей платежной дисциплиой, но люди которые побывали в браке, лучше возвращают долги, чем те кто не состоял в отношениях совсем)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
def relation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

relation('salary_cat')

,debt
salary_cat,
бедный,0.061828
богатый,0.068966
средний,0.081570
выше среднего,0.084435


Вывод:
Люди с доходом более 50 000 руб. чаще просрачивают кредит.

Но если перепроверить данные, построив другую функцию, то получим

In [41]:
print('Разобьем доход на уровни: 0,20-50, 50-100, 100-250, 250-500, 500+')

def income(data):
    if data == 0:
        return '0'
    elif data <= 50000:
        return '20-50'
    elif data <= 100000:
        return '50-100'
    elif data <= 250000:
        return '100-250'
    elif data <= 500000:
        return '250-500'
    else:
        return '500+'
#

data['category_income'] = data['total_income'].apply(income)
data_pivot_income = data.pivot_table(index='category_income', columns='debt', values='total_income', aggfunc='count')
#считаем процент должников
data_pivot_income['ratio %'] = (data_pivot_income[1] / data_pivot_income[0]) * 100
data_pivot_income['ratio %'] = data_pivot_income['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_income['sum'] = data_pivot_income[1] + data_pivot_income[0]

data_pivot_income


Разобьем доход на уровни: 0,20-50, 50-100, 100-250, 250-500, 500+


debt,0,1,ratio %,sum
category_income,,,,
100-250,12985,1193,9,14178
20-50,349,23,6,372
250-500,2411,180,7,2591
50-100,3760,331,8,4091
500+,208,14,6,222


Вывод: люди с доходом 50 000 руб. все еще самые ответственные заемщики, но также к ним прибавились люди с дохожом от 500 000 руб, видимо сказыается цель кредита и т.к. мы выводили таблицу именно по этой категории, я бы выдвинул две гипотезы:
1. в этой категории много пенсионеров, а это чаще люди более ответсвенные в силу своего возраста и воспитания.
2. это люди, берущие займ на недвижимость и не хотят допускать просрочки, т.к. есть риски, например банк может затребовать погасить сразу всю сумму,  что для них будет фатальным, ну и видимо, не хотят портить историю,т.к. скорей всего это не последний кредит в их жизни.)
3. что касатся людей с доходом 500 000+, с таким уровнем ежемесчный платеж комфортный и позволяет не допускать просрочки, возможно стоит автоплатеж.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
data_pivot_purpose = data.pivot_table(index='category_purpose', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose['ratio %'] = (data_pivot_purpose[1] / data_pivot_purpose[0]) * 100
data_pivot_purpose

debt,0,1,ratio %
category_purpose,,,
автомобиль,3903,403,10.325391
недвижимость,10029,782,7.797388
образование,3643,370,10.156464
свадьба,2138,186,8.699719


**Вывод:
Заемщики, берущие кредит для приобретения/проведение операций с жильем, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.**

## Шаг 4. Общий вывод

Отвечая на поставленный вопрос "Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", могу дать следующий ответ - семеное положение и количество детей влияет на факт погашения кредита в строк:
-заемщики с официально оформленными отношениями (или которые в прошлом были в официальном в браке) и не имеющие детей - самые ответственные заемщики;
-заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей -  менее ответственные заемщики.


ТАКЖЕ МОЖНО ДОБАВИТЬ:

Ответственный заещик	Не имеет детей,	находится/был в официальном браке	Уровень дохода < 50000	Берет кредит на жилье или пенсионер, который обладает более лучшей платежной дисциплиной.
Менее ответственный заещик	Имеет 1-2 детей	Не в узаконенных отношениях/не в отношениях	Уровень дохода > 50000	Берет кредит на образование/свадьбу. Также ответственный плательщик с доходом +500 000

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [X]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.